In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from PIL import Image
import os
import torchvision as tv
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import transforms, models
import matplotlib.pyplot as plt
from collections import OrderedDict
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from PIL import Image
import os
import torchvision as tv
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt

In [ ]:

# 训练集预处理
preprocessTrain = transforms.Compose([
    # transforms.Resize([230, 230]),
    transforms.Resize([224, 224]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomRotation(15),
    # 功能：修改修改亮度、对比度和饱和度
    transforms.ColorJitter(brightness=0.5, contrast=0.5,hue=0.5),

    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 测试验证集预处理
preprocessVal_Test = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 从文件中读取数据
def defaultLoader(path,ifTrain):
    img_pil =  Image.open(path).convert('RGB')
    # print(len(img_pil))
    if ifTrain==True:
        img_tensor = preprocessTrain(img_pil)
    else:
        img_tensor = preprocessVal_Test(img_pil)
    return img_tensor


# 从txt中获取类型
def getClassFromTxt(txtPath):
    index_classes={}
    with open(txtPath, 'r', encoding='utf-8') as f:
        txts=f.readlines()
        txts=[t[:-1] for t in txts]
        for t in txts:
            label,name=t.split(' ')[1].split('.')
            index_classes[int(label)-1]=name
    return index_classes


# 获取图片的路径
def getImgPath(imgPath):
    imgs=[]
    for imgName in os.listdir(imgPath):
        imgs.append(imgName)
    return imgs

# 判断样本是否平衡，传入dataloader
def ifBalance(train_loader):
    class_num_list=[0 for i in range(200)]
    for i, (data, labels) in enumerate(train_loader):
        for label in labels:
            class_num_list[label]+=1
    print(class_num_list)


# dataset类
class birdTrainDataSet(Dataset):
    def __init__(self,imgTrainPath,txtClassPath, ifTrain):
        self.class_num_list=[0 for i in range(200)]
        self.txtClassPath=txtClassPath
        self.imgTrainPath=imgTrainPath
        self.ifTrain=ifTrain
        self.index_classes=getClassFromTxt(txtClassPath)
        # print(self.index_classes)
        self.imgNames=getImgPath(imgTrainPath)
        # 统计类的种类
        self.getClassNum()

    def __getitem__(self, index):
        imgName=self.imgNames[index]
        num,name=imgName.split('.',1)
        label=int(num)
        img=defaultLoader(os.path.join(self.imgTrainPath,imgName),self.ifTrain)
        label=label-1
        return img,label

    def __len__(self):
        return len(self.imgNames)

    def getClassNum(self):
        for imgname in self.imgNames:
            num,name=imgname.split('.',1)
            label=int(num)-1
            self.class_num_list[label]+=1
        self.class_num_list=1/np.array(self.class_num_list)

In [ ]:

def getImageNet():
    NetModel = models.densenet121(pretrained=True)
    # print(NetModel.classifier)

    # for param in NetModel.parameters():
    #     param.requires_grad = False
        # print(param.shape)

    classifier = nn.Sequential(OrderedDict([
                            ('Linear1_wbq', nn.Linear(1024, 512)),
                            ('relu', nn.ReLU()),
                            ('relu', nn.Dropout2d()),
                            ('Linear2_wbq', nn.Linear(512, 200)),
                            ]))

    NetModel.classifier = classifier

    # fc_inputs = NetModel.fc.in_features
    # NetModel.fc = nn.Sequential(
    #     nn.Linear(fc_inputs, 512),
    #     nn.ReLU(),
    #     nn.Linear(512, 200),
    # )
    # print(denseNetModel.classifier)

    return NetModel


In [ ]:
if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 获取dataset
    imgTrainPath='../../data/bird/train_set'
    imgValPath='../../data/bird/val_set'
    txtClassPath='../../data/bird/classes.txt'

    bird_data_Trainset=birdTrainDataSet(imgTrainPath,txtClassPath,True)
    bird_data_Valset=birdTrainDataSet(imgValPath,txtClassPath,False)

    # weights=[]
    # for data, label in bird_data_Trainset:
    #     weights.append(bird_data_Trainset.class_num_list[label])

    # # 注意这里的weights应为所有样本的权重序列，其长度为所有样本长度
    # sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(bird_data_Trainset),replacement=True)

    # 获取dataloader
    b_s=16
    trainloader = DataLoader(bird_data_Trainset,batch_size=b_s,num_workers=0)
    # trainloader = DataLoader(bird_data_Trainset,batch_size=b_s,num_workers=2, sampler = sampler)
    valloader = DataLoader(bird_data_Valset,batch_size=b_s,shuffle=False,num_workers=0)

    # 获取预训练的denseNet模型
    denseNetModel = getImageNet()
    # 交叉熵损失函数
    criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(bird_data_Trainset.class_num_list).to(device).float())
    # 优化器
    # optimizer=optim.SGD(denseNetModel.parameters(), lr=0.001, momentum=0.9, factor=0.6)

    lr=0.0015
    optimizer = optim.Adam([
            {"params":denseNetModel.classifier.parameters(),"lr":lr},
            {"params":denseNetModel.features.parameters(),"lr":1e-6},
            ],
            lr=lr, #默认参数
        )

    # optimizer = optim.Adam(filter(lambda p: p.requires_grad, denseNetModel.parameters()),lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

    path='./bird_kaggle_v1.pt'
    path_new='./bird_kaggle_new.pt'

    if os.path.exists(path):
        denseNetModel = torch.load(path)
        print('加载先前模型成功')
    else:
        print('未加载原有模型训练')

    epochNum=20

    # 在内存里面
    maxValAcc=0
    index_to_classes=getClassFromTxt(txtClassPath)
    denseNetModel=denseNetModel.to(device)


    for epoch in range(epochNum):

        denseNetModel.train()
        train_loss=0
        train_correct,train_total=0,0
        train_num=0

        for batch, (data, target) in enumerate(trainloader):
            data=data.to(device)
            target=target.to(device)

            optimizer.zero_grad()
            output = denseNetModel(data)

            prediction = torch.argmax(output, 1)

            batchCorrect = (prediction == target).sum().float()
            batchSize=len(target)

            train_correct += batchCorrect
            train_total += batchSize

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss+=loss.item()
            train_num+=1

            if batch%5==0 and batch!=0:
                print(f'单batch：第{epoch+1}个epoch中训练到第{batch*b_s}个图片,训练集准确率为{100*train_correct/train_total}%')
            
        print('-'*35)
        print(f'epoch：第{epoch+1}次迭代,训练集准确率为{100*train_correct/train_total}%,loss为{train_loss/train_num}')

        # 进行测试
        denseNetModel.eval()
        test_loss=0
        test_correct,test_total=0,0

        valAcc=0
        val_num=0

        with torch.no_grad():
            for batch, (data, target) in enumerate(valloader):
                data=data.to(device)
                target=target.to(device)

                output = denseNetModel(data)
                prediction = torch.argmax(output, 1)
                batchCorrect = (prediction == target).sum().float()
                batchSize=len(target)

                loss = criterion(output, target)
                test_loss+=loss
                val_num+=1
                test_correct += batchCorrect
                test_total += batchSize


        valAcc=100*test_correct/test_total

        print(f'epoch：第{epoch+1}次迭代,验证集准确率为{valAcc}%，loss为{test_loss/val_num}')
        print('-'*35)

        if valAcc > maxValAcc:
            maxValAcc=valAcc
            # 保存模型
            torch.save(denseNetModel, path)
            print()
            print('模型更新成功~')
            print()
        # elif abs(valAcc-maxValAcc)<0.5:
        #   lr=lr*0.8
        #   optimizer = optim.Adam(filter(lambda p: p.requires_grad, denseNetModel.parameters()),lr=lr)
        #   print('优化器更新成功')
        # else:
        #   pass
        scheduler.step()
        # 打印当前学习率
        print('当前学习率为：',end=' ')
        print(optimizer.state_dict()['param_groups'][0]['lr'])

        torch.save(denseNetModel,path_new)